In [1]:
import pandas as pd
import shelve
import pickle
import scipy.io as sio
from scipy.sparse import *

In [2]:
ratingsFrame = pd.read_csv("ml-latest/csv/ratings.csv").loc[:,["userId", "movieId", "rating"]]
moviesFrame = pd.read_csv("ml-latest/csv/movies.csv").loc[:,["movieId"]]

In [21]:
# ratingsFrame.iloc[ratingsFrame.shape[0] - 1]["userId"]
# ratingsFrame.shape
# type(ratingsFrame.loc[1,"rating"])

In [25]:
moviesFrame.iloc[moviesFrame.shape[0] - 1]["movieId"]

176279

In [22]:
ratingsFrame.sort_values(by = 'movieId', ascending=True, inplace=False).iloc[ratingsFrame.shape[0] - 1]["movieId"]

176275.0

In [26]:
rating_matrix_lil = coo_matrix(([], ([], [])), shape=(270897, 176280)).tolil()
for index, row in ratingsFrame.iterrows():
    if index % 100000 == 0:
        print("%.2f%%" % (100 * index / ratingsFrame.shape[0]))  
    rating_matrix_lil[row["userId"], row["movieId"]] = row["rating"]

0.00%
0.38%
0.77%
1.15%
1.54%
1.92%
2.31%
2.69%
3.07%
3.46%
3.84%
4.23%
4.61%
5.00%
5.38%
5.76%
6.15%
6.53%
6.92%
7.30%
7.69%
8.07%
8.45%
8.84%
9.22%
9.61%
9.99%
10.37%
10.76%
11.14%
11.53%
11.91%
12.30%
12.68%
13.06%
13.45%
13.83%
14.22%
14.60%
14.99%
15.37%
15.75%
16.14%
16.52%
16.91%
17.29%
17.68%
18.06%
18.44%
18.83%
19.21%
19.60%
19.98%
20.37%
20.75%
21.13%
21.52%
21.90%
22.29%
22.67%
23.06%
23.44%
23.82%
24.21%
24.59%
24.98%
25.36%
25.75%
26.13%
26.51%
26.90%
27.28%
27.67%
28.05%
28.43%
28.82%
29.20%
29.59%
29.97%
30.36%
30.74%
31.12%
31.51%
31.89%
32.28%
32.66%
33.05%
33.43%
33.81%
34.20%
34.58%
34.97%
35.35%
35.74%
36.12%
36.50%


KeyboardInterrupt: 

In [32]:
save_npz('ml-latest/sparse_matrix_ml-latest.npz', rating_matrix_lil.tocsr())
rating_matrix_lil

<270897x176276 sparse matrix of type '<class 'numpy.float64'>'
	with 26024289 stored elements in LInked List format>

## Tag explore

In [9]:
import shelve
import pickle
d = shelve.open('tags_for_all_movie.pkl', protocol = pickle.HIGHEST_PROTOCOL)
tags_for_all_movie = d['content']
d.close()
movieId_empty = []
for movieId in tags_for_all_movie:
    if(tags_for_all_movie[movieId] == ""):
        movieId_empty.append(int(movieId))
len(movieId_empty)

4769

In [10]:
rating_matrix_csr[:,movieId_empty]

<270897x4769 sparse matrix of type '<class 'numpy.float64'>'
	with 26394 stored elements in Compressed Sparse Row format>

In [11]:
rating_matrix_csr[:,135145]

<270897x1 sparse matrix of type '<class 'numpy.float64'>'
	with 27 stored elements in Compressed Sparse Row format>

## Eliminate item with scarce info

### preview movies without ratings in rating dataset

In [3]:
movieId_list = list(moviesFrame.as_matrix().T[0])
movieId_list.remove(176279)

In [4]:
rating_matrix_csc = load_npz('ml-latest/sparse_matrix_ml-latest.npz').tocsc()
rating_matrix_csc = rating_matrix_csc[:, movieId_list]
rating_matrix_csc

<270897x45842 sparse matrix of type '<class 'numpy.float64'>'
	with 26024289 stored elements in Compressed Sparse Column format>

In [5]:
map_list = list(set(find(rating_matrix_csc)[1]))
map_list.sort()

In [6]:
movieId_list = moviesFrame.as_matrix().T[0]
del_list = list(movieId_list[list(set(list(range(0, 45842))) - set(map_list))])
del_list.append(176279)
len(del_list)

728

In [9]:
d = shelve.open('ml-latest/delete_movies_id_20M/delete_20M.pkl', protocol = pickle.HIGHEST_PROTOCOL)
deleteID_20m = d['content']
deleteID_20m = list(map(int, deleteID_20m))
d.close()
list(set(deleteID_20m) & set(del_list))

[151111, 112648, 169384, 169612, 169614, 169362, 169620, 169562, 104095]

In [124]:
d = shelve.open('ml-latest/empty_movies', protocol = pickle.HIGHEST_PROTOCOL)
d['content'] = del_list
d.close()

### delete specified

In [111]:
rating_matrix_lil = load_npz('ml-latest/sparse_matrix_ml-latest.npz').tolil()
rating_matrix_lil

<270897x176276 sparse matrix of type '<class 'numpy.float64'>'
	with 26024289 stored elements in LInked List format>

In [112]:
d = shelve.open('ml-latest/delete_movies_id_20M/delete_20M.pkl', protocol = pickle.HIGHEST_PROTOCOL)
deleteID_20m = d['content']
d.close()
deleteID_20m = list(map(int, deleteID_20m))
rating_matrix_lil[:,deleteID_20m]

<270897x644 sparse matrix of type '<class 'numpy.float64'>'
	with 1773 stored elements in LInked List format>

In [113]:
rating_matrix_lil[:,deleteID_20m] = 0
rating_matrix_lil

<270897x176276 sparse matrix of type '<class 'numpy.float64'>'
	with 26022516 stored elements in LInked List format>

### compress matrix

In [114]:
map_list = list(set(find(rating_matrix_lil)[1]))
map_list.sort()

In [115]:
del rating_matrix_lil
rating_matrix_csc = load_npz('ml-latest/sparse_matrix_ml-latest.npz').tocsc()
rating_matrix_csc

<270897x176276 sparse matrix of type '<class 'numpy.float64'>'
	with 26024289 stored elements in Compressed Sparse Column format>

In [116]:
rating_matrix_csc = rating_matrix_csc[1:,map_list]
rating_matrix_csc

<270896x44480 sparse matrix of type '<class 'numpy.float64'>'
	with 26022516 stored elements in Compressed Sparse Column format>

In [118]:
save_npz('ml-latest/sparse_matrix_ml-latest_selected.npz', rating_matrix_csc)
d = shelve.open('ml-latest/map_list_ml-latest', protocol = pickle.HIGHEST_PROTOCOL)
d['content'] = map_list
d.close()

## convert to mat

In [48]:
rating_matrix_csc = load_npz('ml-latest/sparse_matrix_ml-latest_selected.npz').tocsc()
sio.savemat('../Step 2 - User Clustering/data/sparse_matrix_ml-latest.mat', {'UI_matrix': rating_matrix_csc})

In [50]:
d = shelve.open('ml-latest/map_list_ml-latest', protocol = pickle.HIGHEST_PROTOCOL)
map_list = d['content'] 
d.close()
sio.savemat('../Step 2 - User Clustering/data/map_list_ml-latest.mat', {'map_list': map_list})